# Pandas a RDBMS
---

In [ ]:
import os

try:
    os.remove('data/database.db')  # Czyszczenie starej bazy
except:
    pass

In [ ]:
import sqlite3

conn = sqlite3.connect('data/database.db')

In [ ]:
conn.execute('''CREATE TABLE COMPANY
         (ID INTEGER PRIMARY KEY     NOT NULL,
         NAME           TEXT    NOT NULL,
         CREATED DATETIME DEFAULT CURRENT_TIMESTAMP
         );'''
    );


In [ ]:
conn.execute('''CREATE TABLE EMPLOYEE
         (ID INTEGER PRIMARY KEY     AUTOINCREMENT,
         NAME           TEXT    NOT NULL,
         AGE            INT     NOT NULL,
         SALARY         REAL,
         COMPANY_ID     INT     NOT NULL,
         CREATED DATETIME DEFAULT CURRENT_TIMESTAMP,
         FOREIGN KEY(COMPANY_ID) REFERENCES COMPANY(ID)
         );'''
    );

In [ ]:
conn.execute("INSERT INTO COMPANY (ID,NAME) VALUES (1, 'ACME')");
conn.execute("INSERT INTO COMPANY (ID,NAME) VALUES (2, 'COLA')");
conn.execute("INSERT INTO COMPANY (ID,NAME) VALUES (3, 'TOYS')");

In [ ]:
conn.execute("INSERT INTO EMPLOYEE (NAME,AGE,SALARY,COMPANY_ID) VALUES ('John D.', 40, 5000.0, 1)");
conn.execute("INSERT INTO EMPLOYEE (NAME,AGE,SALARY,COMPANY_ID) VALUES ('Bob B.', 35, 4500.0, 1)");
conn.execute("INSERT INTO EMPLOYEE (NAME,AGE,SALARY,COMPANY_ID) VALUES ('Alice B.', 50, 6000.0, 1)");
conn.execute("INSERT INTO EMPLOYEE (NAME,AGE,SALARY,COMPANY_ID) VALUES ('Jane G.', 35, 5000.0, 2)");
conn.execute("INSERT INTO EMPLOYEE (NAME,AGE,SALARY,COMPANY_ID) VALUES ('Alan T.', 50, 6000.0, 2)");

In [ ]:
cursor = conn.execute("SELECT id, name from COMPANY")
for row in cursor:
    print(row)

In [ ]:
cursor = conn.execute("SELECT id, name, age, salary, company_id from EMPLOYEE")
for row in cursor:
    print(row)

In [ ]:
cursor = conn.execute('''
SELECT 
    * 
FROM  
    COMPANY 
LEFT JOIN 
    EMPLOYEE 
ON 
    COMPANY.ID==EMPLOYEE.COMPANY_ID
''')
for row in cursor:
    print(row)

---
## Pandas a relacyjne bazy danych

In [ ]:
import pandas as pd
import numpy as np

query = '''
SELECT 
   c.id as cid, 
   c.name as cname, 
   c.created as ccreated, 
   e.id as eid, 
   e.name as ename, 
   age,
   salary, 
   e.created as ecreated 
FROM
   COMPANY as C 
LEFT JOIN 
   EMPLOYEE as E 
ON 
   C.ID==E.COMPANY_ID
'''

df = pd.read_sql(query, conn)
df

In [ ]:
df.dtypes

In [ ]:
df = pd.read_sql(query, conn, parse_dates=['ccreated', 'ecreated'])
df

In [ ]:
df.dtypes

In [ ]:
df = pd.DataFrame({'Student': {0: 'Kowalski J.', 1: 'Nowak A.', 2: 'Korzycki M.'},
                   'WuEf': {0: 5, 1: 4, 2: 2},
                   'Polski': {0: 4, 1: 4, 2: 2},
                   'Matma': {0: 5, 1: 3, 2: 2}})
df1 = pd.melt(df, id_vars=['Student'], value_vars=['WuEf', 'Matma', 'Polski'],
       var_name='Przedmiot', value_name='Ocena')
df1

In [ ]:
df1.to_sql('oceny', conn)

In [ ]:
cursor = conn.execute("SELECT * from oceny")
for row in cursor:
    print(row)

In [ ]:
df1.to_sql('oceny', conn, if_exists='append')

In [ ]:
cursor = conn.execute("SELECT * from oceny")
for row in cursor:
    print(row)

In [ ]:
df1.to_sql('oceny', conn, if_exists='replace')
cursor = conn.execute("SELECT * from oceny")
for row in cursor:
    print(row)

---
## Odpowiedniki SQL w Pandasie

### `SELECT`

In [ ]:
employee = pd.read_sql('SELECT id, name, age, salary, company_id FROM EMPLOYEE', conn)
employee

In [ ]:
pd.read_sql('SELECT id, name, age, salary FROM EMPLOYEE', conn)

In [ ]:
employee[['ID', 'NAME', 'AGE', 'SALARY']]

### `WHERE`

In [ ]:
pd.read_sql(
'''
SELECT 
   id, name, age, salary, company_id 
FROM 
   EMPLOYEE 
WHERE 
   AGE>40
''', conn)

In [ ]:
employee[employee["AGE"] > 40]

### `GROUP BY`

In [ ]:
pd.read_sql('''
SELECT
    company_id, count(*) 
FROM 
    EMPLOYEE 
GROUP BY COMPANY_ID
''', conn)

In [ ]:
pd.DataFrame(employee.groupby('COMPANY_ID')["NAME"].count()).reset_index()

### `JOIN`

In [ ]:
company = pd.read_sql('SELECT id, name FROM COMPANY', conn)
company

In [ ]:
query = '''
SELECT 
   C.id as ID_COMPANY,
   c.name as NAME_COMPANY,  
   e.id as ID_EMPLOYEE, 
   e.name as NAME_EMPLOYEE, 
   age,
   salary, 
   company_id
FROM
   COMPANY as C 
LEFT JOIN 
   EMPLOYEE as E 
ON 
   C.ID==E.COMPANY_ID
'''
pd.read_sql(query, conn)

In [ ]:
company.merge(employee, left_on='ID', right_on='COMPANY_ID', suffixes=("_COMPANY","_EMPLOYEE"))

---
## Dodatkowa sztuczka

### `SQLITE` w pamięci !

In [ ]:
import sqlite3

conn = sqlite3.connect(':memory:')

---
## Porównanie `SQL` vs `Pandas`

### Za Pandasem
- #### W Pandas można łatwo przechodzic <span style="color: cyan">między kolumnami a wierszami</span>
- #### W Pandas możemy mieć <span style="color: cyan">hierarchiczne metadane</span>(`MultiIndex`) w kolumnach i <span style="color: cyan">wierszach</span>
- #### W Pandas format wyjścia jest <span style="color: cyan">elastyczny</span>
- #### Jest wiele narzędzi/bibliotek do przetwarzania danych w Pythonie z których można korzystać

### Za SQL-em
- #### Sprawniejszy przy dużych zbiorach danych 
  - ##### <span style="color: cyan">Dużych</span> - *czyli więcej danych niż mi się mieści w RAM-ie (ang. __Out-of-Core__)*
- #### Sformalizowany, <span style="color: cyan">nieelastyczny format</span>

---

## API `Pandas` dzięki swej popularności stał się pewnego rodzaju standardem który został ponownie użyty w innych bibliotekach. Np.:
- ### [`Polars`](https://www.pola.rs/) - *"szybszy Pandas"* przepisany w języku `Rust`
- ### [`Dask`](https://docs.dask.org/en/stable/dataframe.html) - pandaso-podobna biblioteka do pracy __Out-of-Core__
- ### Honorable mentions:
  - [`Apache Arrow`](https://arrow.apache.org/)
  - [`Koalas`](https://koalas.readthedocs.io/)
  - [`Spark DataFrame`](https://spark.apache.org/)
  - I wiele innych ...